In [2]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

In [4]:
potential = 'QHO'
cutoff = 2

In [5]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [3]:
num_qubits = 2
dev = qml.device("default.qubit", wires=2*num_qubits)

@qml.qnode(dev)
def destructive_swap_test_probs():

    qml.RY(np.pi, wires=[0])
    #qml.RY(np.pi, wires=[1])
    qml.RY(np.pi, wires=[num_qubits])
    #qml.RY(np.pi, wires=[num_qubits+1])
    
    for i in range(num_qubits):
        qml.CNOT(wires=[i, i+num_qubits])    
        qml.Hadamard(wires=i)         

    return qml.probs(wires=range(2*num_qubits))

In [4]:
probs = destructive_swap_test_probs()
probs

array([2.50000000e-01, 0.00000000e+00, 3.74939946e-33, 0.00000000e+00,
       2.50000000e-01, 0.00000000e+00, 3.74939946e-33, 0.00000000e+00,
       2.50000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.50000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [5]:
print(qml.draw(destructive_swap_test_probs)())

0: ──RY(3.14)─╭●──H────┤ ╭Probs
1: ───────────│──╭●──H─┤ ├Probs
2: ──RY(3.14)─╰X─│─────┤ ├Probs
3: ──────────────╰X────┤ ╰Probs


In [6]:
# There are 2^(2*n) possible outcomes.
expected_value = 0
for idx, p in enumerate(probs):

    bitstring = format(idx, '0{}b'.format(2*num_qubits))

    counter_11 = 0
    for i in range(num_qubits):
        a = int(bitstring[i])
        b = int(bitstring[i+num_qubits])
        if (a == 1 and b == 1):
            counter_11 +=1

    print(bitstring, counter_11)
    expected_value += p*(-1)**counter_11

print("Estimated fidelity squared |<ψ|φ>|²:", expected_value)


0000 0
0001 0
0010 0
0011 0
0100 0
0101 1
0110 0
0111 1
1000 0
1001 0
1010 1
1011 1
1100 0
1101 1
1110 1
1111 2
Estimated fidelity squared |<ψ|φ>|²: 0.9999999999999996
